# Example of MD sampling in DNA duplex
*This notebook runs the code for a pre-optimized Cy3+linker pdb, with previously generated Amber FF files. All files are saved in the cy3_full folder.*

*Also shows how the MD runs in an HPC environment can be run*

In [22]:
%load_ext autoreload
%autoreload 2

import os
import sys
# sys.path.append("/path/dyeScreen") # Add package to path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from MD.sampling import scan_DNAconfig, gen_leap_nolinker, modify_mol2
from MD.md_samples import md_run

home = "path/to/package/dyeScreen/"  

# Amber paths
amber_path = "path/to/amber/AmberTools23/bin/" # change if different
path_parmchk2 = amber_path+"parmchk2"

# Define input and output files
path = home + "examples/Cy3/cy3_full/"
samples = "samples/"

leap_out = path + "tleap_screen.in"
dye_mol2 = "cy3-link.mol2"
dye_mol2_rewrite = "cy3-final.mol2"
dye_pdb = path + "cy3-link.pdb"
DNA_pdb =  home + "examples/dna_duplex.pdb"

# Info on the dye molecule (can be extracted from the pdb file)
dye_name = "CY3"
# Atom names for the OPO3H group in the linker
opo3_1 = ['O1','P1','O4','O8','O7','H37']
opo3_2 = ['O2','P2','O3','O5', 'O6', 'H36']

# The atom names that participate in the bonding between the dye and DNA 
attach_cy3 = ['P', "O5'"]*2
attach_dna = ["O3'", "P"]*2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sample dimers in DNA

Sample all possible dimers within DNA scaffold

In [ ]:
# Sample only the positions within 12A of separation and include a box of DX-DNA of 20A
nsamples = scan_DNAconfig(dye_pdb, DNA_pdb, path+samples, resD=1, resL=None, 
                          chainDNA=None, dist_min=12, DNABox=20, DNASt=20,
                          attachment='double', attach_points=[opo3_1,opo3_2], 
                          box_type="doubleAtt") 

Delete and rename atoms from mol2 so they match the pdb file

In [ ]:
modify_mol2([opo3_1, opo3_2], path+dye_mol2, mol2_out=path+dye_mol2_rewrite, 
            attachment='double', parmchk2=path_parmchk2)

Generate input files and run leap 

In [ ]:
gen_leap_nolinker(path+"samples/", amber_path, attach_cy3, attach_dna,
                   path+dye_mol2_rewrite, dye_name=dye_name, wbox=10)

# Run MD trajectories (on HPC)

In [ ]:
from MD.md_samples import cpptraj_check

path = home + 'examples/Cy3/cy3_full/samples/'
amber = "path/to/amber/AmberTools22"

def prefix(nodes, num, amber_p):
    logfile = f'cy3_{num}'
    return f'''#!/bin/sh
#SBATCH --nodes={nodes}
#SBATCH --ntasks={int(32)*nodes}
#SBATCH -J {logfile}
#SBATCH --output={logfile}.log

AMBERPATH="{amber_p}"

# Job Submission
'''

# Check which dimers we actually generated MD input files for
# In the case that some of the samples are invalid, leap won't generate rst7 files.
i_start, i_end = 0, 80
valid_dimers = []
for i in range(i_start, i_end+1):
    dfile = path+f"dimer_{i}_clean.rst7"
    if os.path.exists(dfile):
        valid_dimers.append(i)
print(valid_dimers)       

''' 
# Command to be run on an HPC environment
for dimer_num in valid_dimers:
    print(dimer_num)
    slurm_prefix = prefix(4, dimer_num, amber) 
    md_run(dimer_num, path, amber_path, sample_frefix='dimer_', pdb=None, param=None, coord=None, 
            utoff=12.0, edges_rest=10.0,
            min_cycles=[2000,2000], eq_runtime=[20,1000], prod_runtime=4000, dt=0.002,
            nodes=1, tasks=32, logfile='', 
            sander_path='srun $AMBERPATH/bin/sander.MPI', slurm_prefix=slurm_prefix)
'''

Analysis of MD trajectories and QM to be implemented soon!